In [16]:
import os
import subprocess

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
import getopt
import shutil

from GenerativeLSTM.model_prediction import model_predictor as pr
from GenerativeLSTM import support_functions as sf
from support_modules import stochastic_model as sm
from support_modules import models_merger as mm

In [17]:
parameters = dict()
column_names = {'Case ID': 'caseid',
                'Activity': 'task',
                'lifecycle:transition': 'event_type',
                'Resource': 'user'}
parameters['one_timestamp'] = False  # Only one timestamp in the log
parameters['include_org_log'] = False
parameters['read_options'] = {

    #Production and Purchasing: "%Y-%m-%d %H:%M:%S%z"
    #RunningExample: "%Y-%m-%d %H:%M:%S%z"
    #ConsultaDataMining201618: "%Y-%m-%d %H:%M:%S%z"

    'timeformat': "%Y-%m-%d %H:%M:%S%Z",
    'column_names': column_names,
    'one_timestamp': parameters['one_timestamp'],
    'filter_d_attrib': False}

#parameters['filename'] = 'ConsultaDataMining201618.xes'
#parameters['filename'] = 'PurchasingExample.xes'
parameters['filename'] = 'RunningExample.xes'
#parameters['filename'] = 'Production.xes'
parameters['input_path'] = 'GenerativeLSTM/input_files'

parameters['sm3_path'] = os.path.join('GenerativeLSTM','external_tools', 'splitminer3', 'bpmtk.jar')
parameters['bimp_path'] = os.path.join('GenerativeLSTM','external_tools', 'bimp', 'qbp-simulator-engine_with_csv_statistics.jar')
parameters['concurrency'] = 0.0
parameters['epsilon'] = 0.5
parameters['eta'] = 0.7

parameters['activity'] = 'pred_log'

#PurchasingExample:
#parameters['folder'] = '20240410_1DE076F0_991C_4663_9471_181266EDA48E'
#Production:
#parameters['folder'] = '20240405_408B76AE_502A_4576_BBDB_D182EA5A47EC'
#RunningExample:
parameters['folder'] = '20240411_C51DBDD1_BEB9_4FD6_B23A_5D34B81DE5E1'
#ConsultaDataMining:
#parameters['folder'] = '20240410_92B1B91F_3AEE_42CB_963F_EA8B4320B30A'
parameters['model_file'] = parameters['filename'].split('.')[0] + '.h5'
parameters['log_name'] = parameters['model_file'].split('.')[0]
parameters['is_single_exec'] = False  # single or batch execution
# variants and repetitions to be tested Random Choice, Arg Max, Rules Based Random Choice, Rules Based Arg Max
parameters['variant'] = 'Rules Based Random Choice'
parameters['rep'] = 1

In [18]:
settings = dict()
settings['timeformat'] = parameters['read_options']['timeformat']
settings['column_names'] = parameters['read_options']['column_names']
settings['one_timestamp'] = parameters['read_options']['one_timestamp']
settings['filter_d_attrib'] = parameters['read_options']['filter_d_attrib']

settings['file'] = parameters['filename'].split('.')[0]
settings['sm3_path'] = parameters['sm3_path']

settings['bimp_path'] = parameters['bimp_path']
settings['concurrency'] = parameters['concurrency']
settings['epsilon'] = parameters['epsilon']
settings['eta'] = parameters['eta']

settings['log_path'] = os.path.join('GenerativeLSTM','input_files', settings['file'] + '.xes')
settings['log_path_tobe'] = os.path.join('GenerativeLSTM','input_files', 'spmd', settings['file'] + '.xes')
settings['tobe_bpmn_path'] = os.path.join('GenerativeLSTM','input_files', 'spmd', settings['file'] + '.bpmn')

spmd_tobe = sm.StochasticModel(settings)

Reading log traces ...      [DONE]
 -- Mining Process Structure --
Loading of bpmn structure from file 100.0%...      [DONE]
Counting branch cases...


100%|██████████| 4/4 [00:00<?, ?it/s]


In [19]:
settings['log_path'] = os.path.join('GenerativeLSTM','input_files', settings['file'] + '.xes')
settings['log_path_tobe'] = os.path.join('GenerativeLSTM','input_files', settings['file'] + '.xes')
spmd_asis = sm.StochasticModel(settings)

Reading log traces ...      [DONE]
 -- Mining Process Structure --
Loading of bpmn structure from file 100.0%...      [DONE]
Counting branch cases...


100%|██████████| 4/4 [00:00<?, ?it/s]


In [22]:
spmd_tobe.lrs.branching_probs_tasks

{('Task A', 'Task B'): 1,
 ('Task A', 'End'): 4,
 ('Task D', 'Task E'): 2,
 ('Task D', 'Task A'): 4}

In [24]:
spmd_asis.lrs.branching_probs_tasks

{('Task D', 'Task E'): 2,
 ('Task D', 'Task A'): 4,
 ('Task A', 'Task B'): 1,
 ('Task A', 'End'): 4}

In [5]:
settings = dict()
settings['file'] = parameters['filename'].split('.')[0]
settings['bimp_path'] = parameters['bimp_path']
settings['tobe_bpmn_path'] = os.path.join('GenerativeLSTM','input_files', 'spmd', settings['file'] + '.bpmn')
settings['asis_bpmn_path'] = os.path.join('GenerativeLSTM','input_files', 'simod', settings['file'] + '.bpmn')
settings['csv_output_path'] = os.path.join('GenerativeLSTM','output_files', 'simulation_stats', settings['file'] + '.csv')
settings['output_path'] = os.path.join('GenerativeLSTM','output_files', 'simulation_files', settings['file'] + '.bpmn')
settings['lrs'] = spmd.lrs

mod_mer = mm.MergeModels(settings)

Simulation was successfully executed
